### Initialization

In [ ]:
import numpy as np
import time
import plotly.graph_objects as go


In [ ]:
def get_EU(X, p, alpha):
    return p*(X**alpha)

def get_pk(X_ref, p_ref, X_lot, p_lot, alpha, beta):
    # get the probability of choosing lotery, from a set of X, p, and alpha and beta parameters
    EUr = get_EU(X_ref, p_ref, alpha)
    EUl = get_EU(X_lot, p_lot, alpha)
    # changed the minus sign inside the exponent.
    # standard logistic model, and simplify expressions (same result)
    return 1/(1 + np.exp(-beta*(EUl - EUr))), EUr, EUl

def get_choices(X_ref, p_ref, X_lot, p_lot, alpha, beta):
    pk, EUr, EUl = get_pk(X_ref, p_ref, X_lot, p_lot, alpha, beta)
    choices = np.random.uniform(size=EUl.shape)<pk
    return choices, pk, EUr, EUl

def get_nll_standard(X_ref, p_ref, X_lot, p_lot, choices, alpha, beta):
    ''' compute neg-log-likelihood for a single experiment
    inputs are:
      - the parameters of the task (values and probabilities of lottery and reference)
      - the subject's choices
      - a guess on the subject parameteres (alpha and beta)
    It uses the whole formula, which might be "numerically unstable" because of divisions by 0
    '''
    EUr = p_ref*X_ref**alpha
    EUl = p_lot*X_lot**alpha
    chose_ref = choices==False
    ll_v = 1/(1 + np.exp(-beta*(EUl - EUr)))
    ll_v[chose_ref] = 1 - ll_v[chose_ref]
    return -np.sum(np.log(ll_v))

def get_nll(X_ref, p_ref, X_lot, p_lot, choices, alpha, beta):
    ''' compute neg-log-likelihood for a single experiment
    inputs are:
      - the parameters of the task (values and probabilities of lottery and reference)
      - the subject's choices
      - a guess on the subject parameteres (alpha and beta)
    
    Here we don't compute the likelihood with the whole logistic formula: pk = 1/(1+exp(-beta*(EUl-EUr))
    We can use logarithmic properties to get rid of divisions (which I think is the point of using the -log-likelihood instead of the likelihood)
    For the pos. choices, we use log(1/whatever) = log(1) - log(whatever) = -log(whatever)
    For the neg. choices, we use log(1 - 1/(1+exp(y))) = log(exp(y)/(1+exp(y))) = y - log(1+exp(y))
    Which is y + (thing_with_pos_choice)
    As there are no divisions, no div/0 expected, but there still might be infinites (overflow) ¯\_(ツ)_/¯
    '''
    # probably this is not complex enough to need its own function, save time in function calling
    EUr = p_ref*X_ref**alpha 
    EUl = p_lot*X_lot**alpha

    # Compute things once, sign flip to possibly save one operation
    y = beta*(EUr - EUl)
    chose_ref = choices==False

    # values to be summed up, already negative:
    nll_v = np.log(1 + np.exp(y))
    nll_v[chose_ref] = nll_v[chose_ref] - y[chose_ref]
    return np.sum(nll_v)

def optimize_brute_force(X_ref, p_ref, X_lot, p_lot, choices, alpha_range, beta_range, steps):
    ''' compute -log-likelihood alpha/beta values within a range and with a resolution
    return the alpha, beta and -log-likelihood grids
    '''
    alpha_vals = np.linspace(alpha_range[0], alpha_range[1], steps)
    beta_vals = np.linspace(beta_range[0], beta_range[1], steps)

    (alpha_grid, beta_grid) = np.meshgrid(alpha_vals, beta_vals)
    alphabeta_shape = alpha_grid.shape
    alpha_grid = alpha_grid.flatten()
    beta_grid = beta_grid.flatten()

    nll = [get_nll(X_ref, p_ref, X_lot, p_lot, choices, alpha_grid[_], beta_grid[_]) 
           for _ in np.arange(len(alpha_grid))]
    alpha_grid = alpha_grid.reshape(alphabeta_shape)    
    beta_grid = beta_grid.reshape(alphabeta_shape)    
    nll = np.asarray(nll).reshape(alphabeta_shape)
    
    return alpha_grid, beta_grid, nll

def optimize_brute_force_standard(X_ref, p_ref, X_lot, p_lot, choices, alpha_range, beta_range, steps):
    ''' try all alpha/beta values within a range and with a resolution
    return the alpha, beta and neg-log-likelihood grids
    '''
    alpha_vals = np.linspace(alpha_range[0], alpha_range[1], steps)
    beta_vals = np.linspace(beta_range[0], beta_range[1], steps)

    (alpha_grid, beta_grid) = np.meshgrid(alpha_vals, beta_vals)
    alphabeta_shape = alpha_grid.shape
    alpha_grid = alpha_grid.flatten()
    beta_grid = beta_grid.flatten()

    nll = [get_nll_standard(X_ref, p_ref, X_lot, p_lot, choices, alpha_grid[_], beta_grid[_]) 
           for _ in np.arange(len(alpha_grid))]
    alpha_grid = alpha_grid.reshape(alphabeta_shape)    
    beta_grid = beta_grid.reshape(alphabeta_shape)    
    nll = np.asarray(nll).reshape(alphabeta_shape)
    
    return alpha_grid, beta_grid, nll


### Test on stability with a single "subject"

In [ ]:
# subject parameters
alpha = 0.7
beta = 0.8

# task parameters
X_ref = 20
p_ref = 1
X_vals = [20, 30, 40, 70, 100]
p_vals = [.1, .25, .5, .75, .9]
trialsPerComb = 6

# get all combinations for trials
(X_grid, p_grid) = np.meshgrid(X_vals, p_vals)
X_cmb_unique = np.tile(X_grid.flatten(), trialsPerComb)
p_cmb_unique = np.tile(p_grid.flatten(), trialsPerComb)
X_lot_trials = np.tile(X_cmb_unique, trialsPerComb)
p_lot_trials = np.tile(p_cmb_unique, trialsPerComb)


choices, pk, EUr, EUl = get_choices(X_ref, p_ref, X_lot_trials, p_lot_trials, alpha, beta)

#print(choices)
# print(X_cmb[choices==False])
nll_hard = get_nll_standard(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha, beta)
nll_smart = get_nll(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha, beta)

print(f'The standard way computed {nll_hard}\nThe "smart" way computed {nll_smart}')

start_time = time.time()
nll_hard = get_nll_standard(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha, beta)
end_time = time.time()
time_hard = end_time-start_time

start_time = time.time()
nll_smart = get_nll(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha, beta)
end_time = time.time()
time_smart = end_time-start_time

print(f'The standard way took {time_hard}\nThe "smart" way took {time_smart}')


# test numerical stability
print('testing stability...')
alpha_range = (0.1, 2)
beta_range = (-4, 4)
steps = 100
print('  optimized way')
alpha_grid, beta_grid, nll = optimize_brute_force(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha_range, beta_range, steps)
print('  standard way')
alpha_grid, beta_grid, nll_standard = optimize_brute_force_standard(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha_range, beta_range, steps)

is_finite = np.sum(np.isfinite(nll.flatten()))
is_finite_standard = np.sum(np.isfinite(nll_standard.flatten()))

print('stability test:')
print(f'\tThe standard way got {is_finite_standard} finite values ({is_finite_standard/np.prod(nll.shape)} %)\n\tThe smart way got {is_finite} finite values ({is_finite/np.prod(nll.shape)} %)')

nll_diff = nll - nll_standard
abs_diff = np.abs(nll_diff[np.logical_and(is_finite, is_finite_standard)])
max_abs_diff = np.nanmax(abs_diff)
print(f'max. difference across values: {max_abs_diff}')



# plot the values for which it is numerically stable:
fig = go.Figure(data=go.Heatmap(
                   z=nll,
                   x=alpha_grid[1,:],
                   y=beta_grid[:,1],
                   colorbar={'title':'-log-likelihood'},
                   hoverongaps = False),
                   layout={"template":"plotly_dark"})

fig.add_scatter(x=[alpha], y = [beta], 
                text = ["true value"], textposition="top center", mode='markers+text', textfont={'color':'red'})
fig.update_layout(
    title_text="-log-likelihood (optimized way)",
    xaxis_title="alpha",
    yaxis_title="beta"
)

fig.show()


fig = go.Figure(data=go.Heatmap(
                   z=nll_standard,
                   x=alpha_grid[1,:],
                   y=beta_grid[:,1],
                   colorbar={'title':'-log-likelihood'},
                   hoverongaps = False),
                   layout={"template":"plotly_dark"})

fig.add_scatter(x=[alpha], y = [beta], 
                text = ["true value"], textposition="top center", mode='markers+text', textfont={'color':'red'}, hoverinfo=None)
fig.update_layout(
    title_text="-log-likelihood (standard way)",
    xaxis_title="alpha",
    yaxis_title="beta"
)

fig.show()

### Do the brute force optimization for several alpha/beta combinations

In [ ]:
# Do the brute force optimization for several alpha/beta combinations

# repeat everything for several true values
X_ref = 20
p_ref = 1
X_lot = 30
p_lot = .75

# get all possible combinations
X_vals = [20, 30, 40, 70, 100]
p_vals = [.1, .25, .5, .75, .9]
trialsPerComb = 6

# combinations
(X_grid, p_grid) = np.meshgrid(X_vals, p_vals)
X_cmb_unique = np.tile(X_grid.flatten(), trialsPerComb)
p_cmb_unique = np.tile(p_grid.flatten(), trialsPerComb)
X_lot_trials = np.tile(X_cmb_unique, trialsPerComb)
p_lot_trials = np.tile(p_cmb_unique, trialsPerComb)


alpha_true_vals = np.linspace(-1.5, 1.5, 8)
beta_true_vals = np.linspace(-2, .5, 2)

(alpha_true_grid, beta_true_grid) = np.meshgrid(alpha_true_vals, beta_true_vals)
alphabeta_shape = alpha_true_grid.shape
alpha_true_grid = alpha_true_grid.flatten()
beta_true_grid = beta_true_grid.flatten()

# plot the results
for k1 in np.arange(len(alpha_true_grid)):
    print(f'{(k1+1)}/{len(alpha_true_grid)}')

    print('generating choices...')
    choices, pk, EUr, EUl = get_choices(X_ref, p_ref, X_lot_trials, p_lot_trials, alpha_true_grid[k1], beta_true_grid[k1])
    
    # Optimize (brute force)
    alpha_range = (-2, 2)
    beta_range = (-4, 4)
    steps = 100

    print('running brute force...')
    alpha_grid, beta_grid, nll_smart = optimize_brute_force(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha_range, beta_range, steps)


    (z_min, z_max) = (nll_smart[np.isfinite(nll_smart)].min(), nll_smart[np.isfinite(nll_smart)].max())

    print('plotting...')
    fig = go.Figure(layout={
        "template":"plotly_dark",
        "scene":{
          "camera":{
             "projection":{"type":"orthographic"}
             }
        }
        })

    fig.add_trace(go.Surface(x=alpha_grid, y=beta_grid, z=nll_smart, 
                    surfacecolor=nll_smart,
                    contours = {
                        "z": {"show": True, "start": z_min, "end": z_min + 1000, "size": 50, "color":"gray"}}))
    fig.add_trace(go.Scatter3d(x=(alpha_true_grid[k1], alpha_true_grid[k1]), 
                      y=(beta_true_grid[k1], beta_true_grid[k1]),
                      z=(z_min, z_max), mode='lines'))

    fig.update_scenes(xaxis_title_text='alpha',  
                    yaxis_title_text='beta',  
                    zaxis_title_text='neg-log-likelihood')
    fig.update_layout(title_text=f'(alpha, beta) = {(alpha_true_grid[k1], beta_true_grid[k1])}')

    fig.show()

        


In [ ]:
# Do the brute force optimization for several alpha/beta combinations (only one lottery combination, should be ill-posed)

# repeat everything for several true values
X_ref = 20
p_ref = 1
X_lot = 30
p_lot = .75

# get all possible combinations
X_vals = [40]
p_vals = [.5]
trialsPerComb = 100 # more trials, since we have less data

# combinations
(X_grid, p_grid) = np.meshgrid(X_vals, p_vals)
X_cmb_unique = np.tile(X_grid.flatten(), trialsPerComb)
p_cmb_unique = np.tile(p_grid.flatten(), trialsPerComb)
X_lot_trials = np.tile(X_cmb_unique, trialsPerComb)
p_lot_trials = np.tile(p_cmb_unique, trialsPerComb)


alpha_true_vals = np.linspace(.1, 1.5, 4)
beta_true_vals = np.linspace(-2, .5, 2)

(alpha_true_grid, beta_true_grid) = np.meshgrid(alpha_true_vals, beta_true_vals)
alphabeta_shape = alpha_true_grid.shape
alpha_true_grid = alpha_true_grid.flatten()
beta_true_grid = beta_true_grid.flatten()

# plot the results
for k1 in np.arange(len(alpha_true_grid)):
    print(f'{(k1+1)}/{len(alphabeta_shape)}')

    print('generating choices...')
    choices, pk, EUr, EUl = get_choices(X_ref, p_ref, X_lot_trials, p_lot_trials, alpha_true_grid[k1], beta_true_grid[k1])
    
    # Optimize (brute force)
    alpha_range = (0.1, 2)
    beta_range = (-4, 4)
    steps = 100

    print('running brute force...')
    alpha_grid, beta_grid, nll_smart = optimize_brute_force(X_ref, p_ref, X_lot_trials, p_lot_trials, choices, alpha_range, beta_range, steps)


    (z_min, z_max) = (nll_smart[np.isfinite(nll_smart)].min(), nll_smart[np.isfinite(nll_smart)].max())
    
    print('plotting...')
    fig = go.Figure(layout={
        "template":"plotly_dark",
        "scene":{
          "camera":{
             "projection":{"type":"orthographic"}
             }
        }
        })

    fig.add_trace(go.Surface(x=alpha_grid, y=beta_grid, z=nll_smart, 
                    surfacecolor=nll_smart,
                    contours = {
                        "z": {"show": True, "start": z_min, "end": z_min + 500, "size": 50, "color":"gray"}}))
    fig.add_trace(go.Scatter3d(x=(alpha_true_grid[k1], alpha_true_grid[k1]), 
                      y=(beta_true_grid[k1], beta_true_grid[k1]),
                      z=(z_min, z_max), mode='lines'))

    fig.update_scenes(xaxis_title_text='alpha',  
                    yaxis_title_text='beta',  
                    zaxis_title_text='neg-log-likelihood')
    fig.update_layout(title_text=f'(alpha, beta) = {(alpha_true_grid[k1], beta_true_grid[k1])}')

    fig.show()